### <b> Importing Libraries </b>

In [21]:
import pickle
from torch_geometric.data import DataLoader
import torch_geometric
import torch.nn as nn
import torch
import random

### <b>Graph Datasets</b>

In [121]:
graphs=pickle.load(open("data/graphs.pkl","rb"))

In [122]:
number_range = 671 # 0 to 671 inclusive
set_size = 67

val_class_0 = set(random.sample(range(number_range), set_size))
test_class_0 = set(random.sample(set(range(number_range)) - set(val_class_0), set_size))

val_class_1 = set(random.sample(range(672,len(graphs)), set_size))
test_class_1 = set(random.sample(set(range(672,len(graphs))) - set(val_class_1), set_size))


result_pipe = val_class_0 | val_class_1 | test_class_0 | test_class_1
train_set=set(range(len(graphs)))-result_pipe

len(val_class_0),len(val_class_1),len(test_class_0),len(test_class_1),len(train_set)

C:\Users\rbw19\AppData\Local\Temp\ipykernel_30184\3907160027.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_class_0 = set(random.sample(set(range(number_range)) - set(val_class_0), set_size))
C:\Users\rbw19\AppData\Local\Temp\ipykernel_30184\3907160027.py:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_class_1 = set(random.sample(set(range(672,len(graphs))) - set(val_class_1), set_size))


(67, 67, 67, 67, 1078)

In [123]:
val_data=[graphs[i] for i in val_class_0]+[graphs[i] for i in val_class_1]
test_data=[graphs[i] for i in test_class_0]+[graphs[i] for i in test_class_1]
train_data=[graphs[i] for i in train_set]

In [124]:
train_dataset = DataLoader(train_data, batch_size=10, shuffle=True) # splitting with the batch size of 2
val_dataset= DataLoader(val_data, shuffle=True) 
test_dataset=DataLoader(test_data, shuffle=True)

c:\Users\rbw19\OneDrive\Desktop\GNN_guide\graph_venv\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


### <b> Model creation </b>

In [125]:
# Define a simple Graph Neural Network (GNN) model
class GNNModel(nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(15,8) #equivalent to creating input features with 1D and output features 5D  # https://www.youtube.com/watch?v=2KRAOZIULzw # answers the question how a 1D vector becomes a 5D and 5D becomes 2D
        self.conv2 = torch_geometric.nn.GCNConv(8,3)
        self.conv3 = torch_geometric.nn.GCNConv(3,1)

    def forward(self,batch_data):
        x, edge_index = batch_data.x, batch_data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.sigmoid(x)
        return x

In [127]:

model = GNNModel()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)


model.train()
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  
    for batch in dataset:
        target=batch.y
        optimizer.zero_grad()
        output = model(batch) #will get
        # print("Model Out",output,'Target \n',target)
        # print(output.shape)
        loss = criterion(output.squeeze(), target.float())
        loss.backward()
        optimizer.step()
        # break
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for val_batch in val_dataset:
            # Forward pass for validation
            val_output = model(val_batch)

            # Assuming your model output is a probability score
            # Use BCEWithLogitsLoss if you are dealing with a binary classification problem
            val_predictions = torch.round(torch.sigmoid(val_output))
            
            # Calculate validation accuracy
            labels = val_batch.y.view(-1, 1)
            correct += (val_predictions == labels).sum().item()  #counting out of 10 nodes how many nodes is correctly predicted
            total += labels.size(0)
            # break
            
    # Calculate and print validation accuracy for the current epoch
    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()},Validation Accuracy: {val_accuracy}')
    # break
   


Epoch 1/10, Loss: 0.6931471824645996,Validation Accuracy: 0.7209125475285171
Epoch 2/10, Loss: 0.6931471228599548,Validation Accuracy: 0.7209125475285171
Epoch 3/10, Loss: 0.6931472420692444,Validation Accuracy: 0.7209125475285171
Epoch 4/10, Loss: 0.6931472420692444,Validation Accuracy: 0.7209125475285171
Epoch 5/10, Loss: 0.6931471824645996,Validation Accuracy: 0.7209125475285171
Epoch 6/10, Loss: 0.6931472420692444,Validation Accuracy: 0.7209125475285171
Epoch 7/10, Loss: 0.6931471824645996,Validation Accuracy: 0.7209125475285171
Epoch 8/10, Loss: 0.6931471228599548,Validation Accuracy: 0.7209125475285171
Epoch 9/10, Loss: 0.6931472420692444,Validation Accuracy: 0.7209125475285171
Epoch 10/10, Loss: 0.6931471824645996,Validation Accuracy: 0.7209125475285171


In [118]:
for val_batch in val_dataset:
            # Forward pass for validation
    print(val_batch.x)
#     break

tensor([[3035., 1848.,   73., 3454., 3129., 3077.,  364., 1171., 3763., 2045.,
          536.,  785., 1532., 3771., 2371.],
        [1000., 1727.,   73., 3129., 1852., 1171., 1022., 3763., 2045.,  536.,
         1295., 1172., 1532., 2472., 3794.],
        [3730., 1330.,   73., 1789., 1852., 3077.,  364., 1171., 1022., 3763.,
         2045.,  137., 2495., 2515., 2472.],
        [1286., 3495.,   73., 1851., 1852.,  364., 1171., 1022., 3763., 2045.,
          536., 2319., 3444., 3158., 1532.],
        [3035., 1848.,   73., 2964., 3129., 1852., 3077.,  364., 1171., 1022.,
         2045., 3946., 3158., 1532., 2472.],
        [1193.,  376.,   73., 1789., 1852., 3077.,  364., 1171., 2045.,  536.,
         1799.,  600., 1944., 2472., 1122.],
        [ 493., 3773.,   73., 2588., 3077.,  364., 1171., 1022., 3763.,  536.,
         2195., 2472., 1348., 1654., 3700.],
        [1286., 1848.,   73., 2964., 3129.,  364., 1171., 1022., 3763., 2045.,
          536., 1524., 1533., 1532., 3771.],
        

In [36]:
for batch in dataset:
    print("Batch:")
    print("Batch type:",type(batch))
    print("Number of graphs in the batch:", batch.num_graphs)
    print("Node features shape:", batch.x.shape)
    print("Edge indices shape:", batch.edge_index.shape)
    print("Node labels shape:", batch.y.shape)
    break

Batch:
Batch type: <class 'torch_geometric.data.batch.DataBatch'>
Number of graphs in the batch: 10
Node features shape: torch.Size([100, 15])
Edge indices shape: torch.Size([2, 91])
Node labels shape: torch.Size([100])
